# STM32

The control of the robot - motors and balancing - is delegated to a dedicated STM32. Although the Raspberry PI has certainly sufficient compute power for this function, its Linux operating system is not well suited for real time control. Depending on activity, a process may be suspended for a potentially sufficiently long time for the robot to crash. While with light CPU loads this may never happen, there is no guarantee and consequently such two (or multiple) CPU arrangements are typical in control applications where response time is critical. 

## dtoverlay

The STM32 communicates with the Raspberry PI over UART. These need to be enabled.  Login to [Balena](https://www.balena.io/) (this works only with custom installs!) and open the dashboard for your Raspberry PI. 

Choose `Device Conifguration` and change `Define DT overlays` to 

```
"vc4-fkms-v3d", "uart3", "uart4", "gpio-poweroff,gpiopin=16,active_low=1"
``` 

similar to {numref}`Figure %s <dtoverlay>`. The Raspberry PI will reboot for the change to take effect.

```{figure} figures/dtoverlay.png
:width: 500px
:name: dtoverlay

dtoverlay: configure Raspberry PI IO
```

The Raspberry PI 4 pinout is as follows:

```
        TXD RXD CTS RTS     Board Pins
uart0   14  15              8   10
uart1   14  15              8   10
uart2   0   1   2   3       27  28  (I2C)
uart3   4   5   6   7       7   29
uart4   8   9   10  11      24  21  (SPI0)
uart5   12  13  14  15      32  33  (gpio-fan)
```

```{figure} figures/pi4-pinout.png
:width: 800px

Raspberry PI 4 pin assignment
```

## Customize MicroPython

We use a special MicroPython VM with the following customizations:

* STM32 pin assignments
* floats not stored on heap

The STM32 has many powerful peripherals such as quadrature decoders that in this project will be used for measuring the true wheel RPM. Many of these functions are available only on specific pins. {numref}`Figure %s <stm32-pinout>` shows the pin assignment used for this project.

```{figure} figures/stm32_pinout.png
:name: stm32-pinout

STM32 pin assignments (<a href="../../_images/stm32_pinout.pdf">pdf</a>)
```

The second customization changes how floats are stored. Normally, MicroPython stores most objects, e.g. lists, in a special memory section called heap. By default, floating point variables are also allocated on this heap. As more an more objects are allocated (e.g. due to floating point computations), the heap fills up. Whenever this happens, the "garbage collector" frees space allocated by objects that are no longer used. 

The garbage collector takes some time to run, during which normal program execution is suspended. In real time applications, the resulting delay is not acceptable.

There are several options for dealing with this problem. Here we take the approach of "pre-allocating" during initialization all variables stored on the heap. Once robot control starts, no more objects are allocated on the heap. Consequently memory won't run out and the garbage collector will not run (in fact, can be disabled). This pre-allocation does not work for floats. The special MicroPython VM takes care of this by allocating floats "inline", rather than on the heap.

### MicroPython Source

In [ ]:
%%bash

# interpreter
cd $IOT/mp
if [ ! -d micropython ]
then
    git clone git@github.com:micropython/micropython.git
else
    cd micropython
    git checkout master
    git pull
    git merge master
fi

# library
cd $IOT/mp
if [ ! -d micropython-lib ]
then
    git clone git@github.com:micropython/micropython-lib.git
else
    cd micropython-lib
    git checkout master
    git pull
    git merge master
fi

### Compile

In [ ]:
%%service arm32

cd $IOT/mp/micropython/ports/stm32
cp -rf ../../../boards/MOTOR_HAT boards
make submodules
make BOARD=MOTOR_HAT clean
make BOARD=MOTOR_HAT USER_C_MODULES=../../../modules

### Flash

Install stm32 flasher:

In [ ]:
%%bash

cd /tmp
git clone https://git.code.sf.net/p/stm32flash/code stm32flash-code
cd stm32flash-code
sudo make install

In [ ]:
%%host

import stm32
stm32.flash(info_only=True)

### REPL

In [1]:
%connect serial:///dev/ttyAMA1
import sys
print(sys.platform)

Connected to 2c:00:29:00:09:50:52:42:4e:30:39:20 @ serial:///dev/ttyAMA1
pyboard


## STM32 from Pi

`stm32.py` provides a number of convenience functions for accessing the STM32 from the Raspberry PI:

In [1]:
!cat $IOT_PROJECTS/robot/code/rpi/stm32.py

from iot_device.pydevice import Pydevice
from iot_device import DeviceRegistry, RemoteError
from serial import Serial
from gpiozero import LED as Pin
import asyncio, subprocess, os, time


def hard_reset(boot_mode=False):
    """Hard reset STM32. Same as pressing reset button.

    @param boot_mode: bool Start in "dfu" boot-mode (default False).
    """
    with Pin(21) as nrst, Pin(27) as boot0:
        if boot_mode:
            boot0.on()
        else:
            boot0.off()
        time.sleep(0.1)
        nrst.off()
        time.sleep(0.1)
        nrst.on()
        # let boot process finish
        time.sleep(1)

def _flash_bin(address, firmware, dev, info_only):
    """Flash helper. Used by flash method."""
    if info_only:
        cmd = ['stm32flash', dev]
    else:
        cmd = ['stm32flash', '-v', '-S', address, '-w', firmware, dev]
    process = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    stdout, stderr = process.communicate()
    print(stdout.d

In [34]:
%%host

import stm32

cmd = 'print(4+7, end="")'
print(f"exec({cmd}): {stm32.exec(cmd)}")

print(f"supply voltage: {stm32.supply_voltage():.1f}V")

exec(print(4+7, end="")): 11
supply voltage: 10.1V


## Device Configuration

In [1]:
%%writefile $IOT_PROJECTS/devices/robot.yaml

# motor controller etc.
robot-stm32:
    # uid: 1c:00:26:00:09:50:52:42:4e:30:39:20
    uid: 2c:00:29:00:09:50:52:42:4e:30:39:20
    install-dir: /flash
    path: robot/code
    resources:
        - secrets.py:
            path: libs
        - state.py:
            path: robot/code/rpi/robot
            install-dir: /flash/lib
        - bno055:
            path: libs
            install-dir: /flash/lib
        - stm32

# ble remote control
robot-esp32:
    uid: 1c:00:26:02:4e:30:39:20
    path: robot/code
    resources:
        - secrets.py:
            path: libs
        - esp32

Writing /home/iot/iot49.org/docs/projects/devices/robot.yaml


In [1]:
%connect serial:///dev/ttyAMA1
%rsync

Connected to robot-stm32 @ serial:///dev/ttyAMA1
UPDATE  /flash/lib/comm.py


In [ ]:
% required for files to be copied to output directory

![](figures/stm32_pinout.pdf)